In [2]:
import json
from pathlib import Path

file_path = Path("C:/Users/playdata2/Desktop/SKN_3rd/data/youth_policies_filtered_kr_revised.json")

with file_path.open("r", encoding="utf-8") as f:
    policies = json.load(f)

print(len(policies))

3000


In [3]:
import json
from pathlib import Path

path = Path("C:/Users/playdata2/Desktop/SKN_3rd/data/korea-administrative-district.json")

with path.open("r", encoding="utf-8") as f:
    adm_data = json.load(f)

print(adm_data.keys())

dict_keys(['name', 'version', 'url', 'data'])


```
지역 정보를 나타내는 필드
- "주관기관명": "충청북도"
- "등록기관명": "충청북도"
정책을 제공하는 지자체 → 충청북도
하지만 시/군/구 정보는 없음
따라서 지금 상태에서는 지역 정밀 필터링(예: “청주시 정책만 주세요”)이 불가능함

### STEP 1 — 지역 시·도 필드 추출하기

In [4]:
# 시군구 → 시도 매핑
sigungu_to_sido = {}
SIDO_LIST = []  # 시도 전체 자동 생성

for sido_dict in adm_data["data"]:
    for sido, sigungu_list in sido_dict.items():
        SIDO_LIST.append(sido)
        for gungu in sigungu_list:
            sigungu_to_sido[gungu] = sido

print("시도 개수:", len(SIDO_LIST))
print("시군구 개수:", len(sigungu_to_sido))


시도 개수: 17
시군구 개수: 206


In [5]:
# 지역 정보가 있을만한 필드s
def build_region_text(policy):
    fields = [
        policy.get("정책명", ""),
        policy.get("정책설명", ""),
        policy.get("주관기관명", ""),
        policy.get("등록기관명", ""),
        policy.get("상위기관명", ""),
        policy.get("상위등록기관명", ""),
        policy.get("추가자격조건", "")
    ]
    return " ".join(f for f in fields if f)

### 지역 정규화

In [6]:
import re
import unicodedata

# 0. 조직명/불필요 단어 제거
DROP_WORDS = [
    "특별시", "광역시", "특별자치도", "자치도",
    "도청", "시청", "구청", "군청",
    "센터", "본부", "지원센터",
    "정책과", "정책관", "담당관"
    # "국", "실"  # ← 이건 행정구역까지 깨먹을 수 있어서 뺀 상태
]

def clean_org_words(text: str) -> str:
    if not isinstance(text, str):
        return text

    # 유니코드 정규화 + 공백, html 공백 처리
    text = unicodedata.normalize("NFKC", text)
    text = text.replace("\u200b", "")
    text = text.replace("&nbsp;", " ")
    text = re.sub(r"\s+", " ", text)

    for w in DROP_WORDS:
        text = text.replace(w, "")

    text = re.sub(r"\s+", " ", text).strip()
    return text



# 1. 시도명 정규화
def normalize_sido_name(sido: str) -> str:
    if not isinstance(sido, str):
        return sido

    # 1) 기본 정규화: suffix 제거
    base = (
        sido.replace("특별자치도", "")
            .replace("광역시", "")
            .replace("특별시", "")
            .replace("자치도", "")
            .replace("도", "")
    )

    # 2) 예외 처리: 전북 → 전라북도
    if base == "전북":
        return "전라북도"

    # 3) 나머지는 base 그대로 반환
    return base


# 2. 지역 파싱
def parse_region_v4(text, sigungu_to_sido, SIDO_LIST):
    if not text or not isinstance(text, str):
        return None, None, text

    original_text = text
    cleaned = clean_org_words(text)

    # 2) 시군구 후보 추출
    basic = re.findall(r"([가-힣]{2,}(?:시|군|구))", cleaned)
    spaced = [w + suf for w, suf in re.findall(r"([가-힣]{2,})\s*(시|군|구)", cleaned)]
    sigungu_candidates = list(set(basic + spaced))

    # 3) 시군구 매핑
    for g in sigungu_candidates:
        if g in sigungu_to_sido:
            sido = sigungu_to_sido[g]
            return normalize_sido_name(sido), g, cleaned

    # 4) 시도 탐지
    for sido in SIDO_LIST:
        # 텍스트 매칭용 base (전북, 인천, 부산 등)
        base = (
            sido.replace("특별자치도", "")
                .replace("광역시", "")
                .replace("특별시", "")
                .replace("자치도", "")
                .replace("도", "")
        )

        normalized = normalize_sido_name(sido)  # 반환용 (전북 → 전라북도)

        # 텍스트에는 base나 원본(sido)이 들어 있을 수 있음
        if (
            (base and (base in cleaned or base in original_text))
            or (sido and (sido in cleaned or sido in original_text))
        ):
            return normalized, None, cleaned

    # 5) 단어 단위 매칭
    for w in cleaned.split():
        if w in sigungu_to_sido:
            sido = sigungu_to_sido[w]
            return normalize_sido_name(sido), w, cleaned

    return None, None, cleaned


In [6]:
# 전체 정책에 '지역(시도/시군구)' 필드 적용
for p in policies:
    p["region_text"] = build_region_text(p)
    sido, sigungu, cleaned_region_text = parse_region_v4(
    p["region_text"], sigungu_to_sido, SIDO_LIST)
    p["시도"] = sido
    p["시군구"] = sigungu
    p["region_text_cleaned"] = cleaned_region_text

In [7]:
missing = [p for p in policies if p["시도"] is None]
print("지역 판별 실패 개수:", len(missing))

지역 판별 실패 개수: 344


In [8]:
# 지역 판별 실패 데이터 -> 전국으로 처리 => 지역 조건과 무관하게 검색 결과에 포함
for p in policies:
    sido = p["시도"]
    sigungu = p["시군구"]

    if sido is None:    
        p["지역범위"] = "national"
    elif sigungu:
        p["지역범위"] = "local"
    else:
        p["지역범위"] = "sido"

In [9]:
from collections import Counter
Counter(p["지역범위"] for p in policies)

Counter({'sido': 1512, 'local': 1144, 'national': 344})

In [ ]:
import json
from pathlib import Path

# 저장 경로 설정
output_path = Path("C:/Users/playdata2/Desktop/SKN_3rd/data/youth_policies_processed.json")

# policies : 전처리 완료된 리스트(dict)라고 가정
with output_path.open("w", encoding="utf-8") as f:
    json.dump(
        policies,
        f,
        ensure_ascii=False, 
        indent=2            
    )

print("전처리된 정책 데이터 저장 완료:", output_path)

전처리된 정책 데이터 저장 완료: C:\Users\playdata2\Desktop\SKN_3rd\data\youth_policies_processed.json
